In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define a simple dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, word_to_idx):
        self.texts = texts
        self.labels = labels
        self.word_to_idx = word_to_idx

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = [self.word_to_idx[word] for word in self.texts[idx].split()]

        try:
            label = torch.as_tensor(int(self.labels[idx])).clone().detach()
        except ValueError:
            label = torch.as_tensor(0)

        return {'text': torch.LongTensor(text), 'label': label}

def collate_batch(batch):
    texts = [item['text'] for item in batch]
    labels = [item['label'] for item in batch]

    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)

    return {'text': padded_texts, 'label': torch.stack(labels)}

def read_data_from_csv(file_path):
    texts = []
    labels = []
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            parts = line.strip().split(',')
            if len(parts) >= 2:
                texts.append(parts[0])
                labels.append(parts[1].strip('\"'))
    return texts, labels

# Improved Emotion Analysis Model
class ImprovedEmotionAnalysisModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pretrained_embeddings=None):
        super(ImprovedEmotionAnalysisModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(torch.from_numpy(pretrained_embeddings))
            self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]
        lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        return out

# Read data from CSV file
file_path = '/content/gdrive/MyDrive/DS677 - Fall 23 - DL Project - Paresh, Ojaswi, Dinesh/TextAndEmotions.csv'
texts, labels = read_data_from_csv(file_path)

# Create vocabulary and word_to_idx mapping
vocab = set(' '.join(texts).split())
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}

# Update the dataset and pad sequences
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_dataset = EmotionDataset(train_texts, train_labels, word_to_idx)
test_dataset = EmotionDataset(test_texts, test_labels, word_to_idx)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# Model parameters
embedding_dim = 50
hidden_dim = 100
output_dim = len(set(labels))

# Initialize the model and choose the appropriate pre-trained embeddings
# Download GloVe embeddings from https://nlp.stanford.edu/projects/glove/ and adjust the path
# Example code to load GloVe embeddings:
# embeddings_index = {}
# with open('path_to_glove/glove.6B.50d.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs
# embedding_matrix = np.zeros((len(word_to_idx) + 1, embedding_dim))
# for word, i in word_to_idx.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
# model = ImprovedEmotionAnalysisModel(
#     vocab_size=len(word_to_idx) + 1,
#     embedding_dim=embedding_dim,
#     hidden_dim=hidden_dim,
#     output_dim=output_dim,
#     pretrained_embeddings=embedding_matrix,
# ).to(device)

model = ImprovedEmotionAnalysisModel(
    vocab_size=len(word_to_idx) + 1,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
).to(device)

# Training parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 50

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False):
        text, label = batch['text'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        output = model(text)
        loss = criterion(output, label)
        loss.backward()
        # Implement gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')

# Evaluation
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating', leave=False):
        text, label = batch['text'].to(device), batch['label'].to(device)
        output = model(text)
        predictions = torch.argmax(output, dim=1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

accuracy = accuracy_score(all_labels, all_predictions)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/50, Loss: 4.298942142062717


Epoch 2/50, Loss: 2.4066073894500732


Epoch 3/50, Loss: 0.8778071800867716


Epoch 4/50, Loss: 0.5014093981848823


Epoch 5/50, Loss: 0.4190385606553819


...

Epoch 47/50, Loss: 0.02892321317146222


Epoch 48/50, Loss: 0.025212829052988026


Epoch 49/50, Loss: 0.02620111669724186


Epoch 50/50, Loss: 0.029320076832340822


Test Accuracy: 93.75%


In [ ]:
# Sample inputs
sample_inputs = [
    "I love the new feature!",
    "I am satisfied with the customer service.",
    "This movie is not good!",
    "This software have lots of features.",
    "Incredible customer experience!",
    "I am anxious"
]

# Predict emotions for each sample input
for sample_input in sample_inputs:
    # Preprocess the sample input
    sample_input_indices = [word_to_idx[word] for word in sample_input.split() if word in word_to_idx]

    # Check if the sample input is empty after filtering
    if not sample_input_indices:
        print(f"No valid words found in the sample input: {sample_input}")
    else:
        # Convert to tensor and add batch dimension
        sample_input_tensor = torch.LongTensor(sample_input_indices).unsqueeze(0).to(device)

        # Pass through the trained model
        model.eval()
        with torch.no_grad():
            output = model(sample_input_tensor)

        # Interpret the model's output
        predicted_class = torch.argmax(output, dim=1).item()

        # Map the predicted class to the corresponding emotion label
        emotion_labels = {0: "Negative", 1: "Positive"}  # Update with your specific labels
        predicted_emotion = emotion_labels.get(predicted_class, "Unknown")

        print(f"The predicted emotion for the sample input '{sample_input}' is: {predicted_emotion}")


The predicted emotion for the sample input 'I love the new feature!' is: Positive
The predicted emotion for the sample input 'I am satisfied with the customer service.' is: Positive
The predicted emotion for the sample input 'This movie is not good!' is: Positive
The predicted emotion for the sample input 'This software have lots of features.' is: Negative
The predicted emotion for the sample input 'Incredible customer experience!' is: Positive
The predicted emotion for the sample input 'I am anxious' is: Negative
